In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 8s 0us/step


In [0]:
def get_data(n):
    train_dn = os.path.join(dn, n)
    contents = []
    sentiment = []
    pos_fn = os.path.join(train_dn, "pos", "*.txt")
    for fn in glob.glob(pos_fn):
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            sentiment.append(1)

    neg_fn = os.path.join(train_dn, "neg", "*.txt")
    for fn in glob.glob(neg_fn):
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            sentiment.append(0)

    df = pd.DataFrame({
        "content":contents,
        "sentiment":sentiment
    }, columns=["content", "sentiment"])
    return df

In [16]:
import os
import glob
import pandas as pd

dn = os.path.dirname(dataset)
dn = os.path.join(dn, "aclImdb")

train_df = get_data("train")
test_df = get_data("test")
test_df

,content,sentiment
0,Fair and nifty little science fiction/horror f...,1
1,I think Gerard's comments on the doc hit the n...,1
2,This is one of the best Czech movies I have ev...,1
3,"This film, for what it was set out to be, succ...",1
4,"Excellent performance by Mary KAy Place, Steve...",1
...,...,...
24995,First I have to say that I have read everythin...,0
24996,"""The Duke"" is a film based in the heart of the...",0
24997,So we're supposed to find it funny that this w...,0
24998,This show had a promising start as sort of the...,0
